In [2]:
import re
import tweepy
import pandas as pd
from tweepy import OAuthHandler
import yaml
# from http.client import IncompleteRead
# import threading 
# import datetime 
    
    
with open('credential1.yaml') as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)
    
consumer_key = config['consumer_key']
consumer_secret = config['consumer_secret']
access_token = config['access_token']
access_secret = config['access_secret']
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

#### Collect tweets with hashtag `#30mapchallenges`

In [3]:
def collect_tweets(search_words, since):
    #tweets
    tweet_id = []
    tweet_text = []
    language = []
    tweet_created_at = []
    retweet_count = []
    favorite_count= []
    hashtags= []
    #users
    user_id = []
    user_name = []
    user_screen_name = []
    user_loc = []
    user_follower = []
    friends_count = []
    #media
    media = []
    for tweet in tweepy.Cursor(api.search,
                               q=search_words,
                               since= since, 
                               include_entities=True, 
                               tweet_mode="extended").items():
        #tweets
        tweet_id.append(tweet.id)
        tweet_text.append(tweet.full_text)
        language.append(tweet.lang)
        tweet_created_at.append(tweet.created_at)
        retweet_count.append(tweet.retweet_count)
        favorite_count.append(tweet.favorite_count)
        try:
            hashtags.append(tweet.entities.get('hashtags', []))
        except KeyError:
            pass
#         hashtags.append(tweet.entities.hashtags)
        #users
        user_id.append(tweet.user.id)
        user_name.append(tweet.user.name)
        user_screen_name.append(tweet.user.screen_name)
        user_loc.append(tweet.user.location)
        user_follower.append(tweet.user.followers_count)
        friends_count.append(tweet.user.friends_count)
        #media
        try:
            media.append(tweet.entities.get('media')[0].get('media_url'))
#             print(tweet.entities.get('media'))
        except Exception as e:
            media.append(None)
        #print([tweet.user.name,tweet.created_at,tweet.text])
    df = pd.DataFrame({'tweet_id':tweet_id,
                        'tweet_text':tweet_text,
                        'language':language,
                        'tweet_created_at':tweet_created_at,
                        'retweet_count':retweet_count,
                        'favorite_count':favorite_count,
                        'hashtags':hashtags,
                        'user_id':user_id,
                        'user_name':user_name,
                        'user_screen_name':user_screen_name,
                        'user_loc':user_loc,
                        'user_follower':user_follower,
                        'friends_count':friends_count,
                        'media':media})
            
    return df

In [4]:
search_words = "#30DayMapChallenge -filter:retweets"
since = "2020-10-30"

df = collect_tweets(search_words, since)

df = df[(df.tweet_created_at >= '2020-11-02 00:00:00') & 
   (df.tweet_created_at <= '2020-11-03 00:00:00')]

# df1 = df[df['tweet_text'].str.contains("points") | df['tweet_text'].str.contains("day 1")]

df = df[df['media'].notnull()]
df['tweet_url'] = df['tweet_text'].str.extract('(?P<url>https?://[^\s]+)', expand=False).str.strip()


# df.to_csv(f"./data/day2.csv")

In [20]:
df_day1_late = df[df['tweet_text'].str.contains('points', case=False) & df['tweet_text'].str.contains('day 1', case=False)]
# df_day1_late.to_csv(f"./data/day1_late.csv")

## Overview

In [188]:
# df = pd.read_csv("./data/day1.csv")

In [122]:
df.user_loc

163                     
164    Neo SCL / Neo BCN
165                 Wien
166     Montréal, Québec
169                     
             ...        
704                     
705                  y,x
706        Leicester, UK
707                     
708                 Iraq
Name: user_loc, Length: 456, dtype: object

In [121]:
df[['retweet_count','favorite_count', 'user_follower', 'friends_count']].describe()

,retweet_count,favorite_count,user_follower,friends_count
count,456.000000,456.000000,456.000000,456.000000
mean,2.923246,14.309211,1602.668860,668.684211
std,5.005121,30.491772,6699.908748,852.130466
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,3.000000,69.000000,154.750000
50%,1.000000,7.000000,269.000000,361.000000
75%,3.000000,16.000000,844.000000,831.500000
max,82.000000,546.000000,80339.000000,5446.000000


## Top favorited `#30mapchallenges` post

In [6]:
df_fav15 = df.sort_values(by='favorite_count', ascending=False).head(15)
df_fav15 = df_fav15[['user_screen_name', 'retweet_count', 'favorite_count', 'media']]
df_fav15['user_screen_name'] = '**[@' + df['user_screen_name'] +']('+ df['tweet_url']+')**'
df_fav15['media'] = '![](' + df['media'] +')'
df_fav15

,user_screen_name,retweet_count,favorite_count,media
888,**[@rajbhagatt](https://t.co/LkmRQdH81P)**,91,622,![](http://pbs.twimg.com/media/El0q6kyU8AAHlSS...
725,**[@JoeWDavies](https://t.co/mwHzmibvom)**,24,159,![](http://pbs.twimg.com/media/El1571EXgAAMlJJ...
702,**[@sarahbellmaps](https://t.co/tOxg3nHbbu)**,14,149,![](http://pbs.twimg.com/media/El2ACUrVcAAe5Mr...
744,**[@darenw](https://t.co/DNoJo2SWsA)**,11,119,![](http://pbs.twimg.com/ext_tw_video_thumb/13...
726,**[@SouthArrowMaps](https://t.co/pdQa0Mdjhs)**,14,102,![](http://pbs.twimg.com/media/ElkWSGsUcAAvE08...
921,**[@jakub_nowosad](https://t.co/qnzLDxIOjC.)**,30,99,![](http://pbs.twimg.com/media/El0ZGvdW0AAEeQA...
1095,**[@darenw](https://t.co/rZ2gtbJmI4)**,4,95,![](http://pbs.twimg.com/media/Elyc4d4XUAEs09h...
805,**[@RichardZimerman](https://t.co/vCXHLLZdsl)**,9,89,![](http://pbs.twimg.com/media/El1P1uCXYAI2ZmF...
1011,**[@gontsa](https://t.co/bOaU5zS5BG)**,23,86,![](http://pbs.twimg.com/media/ElzopFSXUAAHiXU...
645,**[@Asgerpetersen](https://t.co/9npzOdq8gs)**,3,78,![](http://pbs.twimg.com/media/El2YONXW0AEYxvL...


In [7]:
from tabulate import tabulate

print(tabulate(df_fav15, headers='keys', tablefmt='github'))

|      | user_screen_name                                |   retweet_count |   favorite_count | media                                                                                        |
|------|-------------------------------------------------|-----------------|------------------|----------------------------------------------------------------------------------------------|
|  888 | **[@rajbhagatt](https://t.co/LkmRQdH81P)**      |              91 |              622 | ![](http://pbs.twimg.com/media/El0q6kyU8AAHlSS.jpg)                                          |
|  725 | **[@JoeWDavies](https://t.co/mwHzmibvom)**      |              24 |              159 | ![](http://pbs.twimg.com/media/El1571EXgAAMlJJ.jpg)                                          |
|  702 | **[@sarahbellmaps](https://t.co/tOxg3nHbbu)**   |              14 |              149 | ![](http://pbs.twimg.com/media/El2ACUrVcAAe5Mr.jpg)                                          |
|  744 | **[@darenw](https://t.co/DNoJo2SWsA)

## Top retweeted `#30mapchallenges` post

In [8]:
df_rt15 = df.sort_values(by='retweet_count', ascending=False).head(15)
df_rt15 = df_rt15[['user_screen_name', 'retweet_count', 'favorite_count', 'media']]
df_rt15['user_screen_name'] = '**[@' + df['user_screen_name'] +']('+ df['tweet_url']+')**'
df_rt15['media'] = '![](' + df['media'] +')'
df_rt15

,user_screen_name,retweet_count,favorite_count,media
888,**[@rajbhagatt](https://t.co/LkmRQdH81P)**,91,622,![](http://pbs.twimg.com/media/El0q6kyU8AAHlSS...
921,**[@jakub_nowosad](https://t.co/qnzLDxIOjC.)**,30,99,![](http://pbs.twimg.com/media/El0ZGvdW0AAEeQA...
725,**[@JoeWDavies](https://t.co/mwHzmibvom)**,24,159,![](http://pbs.twimg.com/media/El1571EXgAAMlJJ...
1011,**[@gontsa](https://t.co/bOaU5zS5BG)**,23,86,![](http://pbs.twimg.com/media/ElzopFSXUAAHiXU...
980,**[@lecartographe](https://t.co/xbM3Hnj76T)**,23,54,![](http://pbs.twimg.com/media/Elz6jqVXYAEwIts...
922,**[@Juanizio_C](https://t.co/VhjWgGgXQJ)**,22,63,![](http://pbs.twimg.com/media/El0ZtfAWMAEgoNZ...
1116,**[@gonzalotalavera](https://t.co/I6UNoGYVOx)**,22,60,![](http://pbs.twimg.com/media/ElyMGZFWkAA7Fx2...
611,**[@ZorzalErrante](https://t.co/8oGAs84EF6)**,16,75,![](http://pbs.twimg.com/media/El2yW6KXgAUOs9D...
759,**[@moaimx](https://t.co/fw1TQT6FGa)**,15,66,![](http://pbs.twimg.com/media/El1n-ufWMAAV9yB...
702,**[@sarahbellmaps](https://t.co/tOxg3nHbbu)**,14,149,![](http://pbs.twimg.com/media/El2ACUrVcAAe5Mr...


In [9]:
print(tabulate(df_rt15, headers='keys', tablefmt='github'))

|      | user_screen_name                                |   retweet_count |   favorite_count | media                                               |
|------|-------------------------------------------------|-----------------|------------------|-----------------------------------------------------|
|  888 | **[@rajbhagatt](https://t.co/LkmRQdH81P)**      |              91 |              622 | ![](http://pbs.twimg.com/media/El0q6kyU8AAHlSS.jpg) |
|  921 | **[@jakub_nowosad](https://t.co/qnzLDxIOjC.)**  |              30 |               99 | ![](http://pbs.twimg.com/media/El0ZGvdW0AAEeQA.jpg) |
|  725 | **[@JoeWDavies](https://t.co/mwHzmibvom)**      |              24 |              159 | ![](http://pbs.twimg.com/media/El1571EXgAAMlJJ.jpg) |
| 1011 | **[@gontsa](https://t.co/bOaU5zS5BG)**          |              23 |               86 | ![](http://pbs.twimg.com/media/ElzopFSXUAAHiXU.png) |
|  980 | **[@lecartographe](https://t.co/xbM3Hnj76T)**   |              23 |               54 | ![](

### Send tweet via API

In [3]:
api.update_status("Look, I'm tweeting from #Python")


Status(_api=<tweepy.api.API object at 0x0000011E60202A48>, _json={'created_at': 'Mon Nov 02 22:53:36 +0000 2020', 'id': 1323397878983856130, 'id_str': '1323397878983856130', 'text': "Look, I'm tweeting from #Python", 'truncated': False, 'entities': {'hashtags': [{'text': 'Python', 'indices': [24, 31]}], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="https://hn303.github.io/CamLandEc-RM03/" rel="nofollow">Lonon_REST</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 528087151, 'id_str': '528087151', 'name': 'Haifeng Niu', 'screen_name': 'niu_haifeng', 'location': 'Cambridge, England', 'description': 'PhD candidate in Lab of Interdisciplinary Spatial Analysis, Department of Land Economy, University of Cambridge @CambridgeLandEc', 'url': 'https://t.co/nuaPhizg9Y', 'entities': {'url': {'urls': [{'url': 'https://t.co/nuaPhizg9Y', 'expanded_